In [4]:
from opentrons import simulate
from opentrons.drivers.rpi_drivers import gpio
import time
import math

ctx = simulate.get_protocol_api('2.1')

buffer_rack = ctx.load_labware('opentrons_6_tuberack_falcon_50ml_conical', '10',
        'buffer labware')
dest_racks = [
            ctx.load_labware('opentrons_24_tuberack_generic_2ml_screwcap', slot,
                            'Destination tubes labware ' + str(i+1))
            for i, slot in enumerate(['4', '1', '5', '2'])
    ]

/home/smonzon/.opentrons/deck_calibration.json not found. Loading defaults
/home/smonzon/.opentrons/robot_settings.json not found. Loading defaults


In [11]:
bf_tubes = buffer_rack.wells()[:4]
NUM_SAMPLES = 96

number_racks = math.ceil(NUM_SAMPLES/len(dest_racks[0].wells()))

dest_sets = [
        [tube
         for rack in dest_racks
         for tube in rack.wells()
        ][:NUM_SAMPLES][i*len(dest_racks[0].wells()):(i+1)*len(dest_racks[0].wells())]
        for i in range(number_racks)
        ]

for bf_tube,dest in zip(bf_tubes,dest_sets):
    print(bf_tube)
    print(dest)

A1 of buffer labware on 10
[A1 of Destination tubes labware 1 on 4, B1 of Destination tubes labware 1 on 4, C1 of Destination tubes labware 1 on 4, D1 of Destination tubes labware 1 on 4, A2 of Destination tubes labware 1 on 4, B2 of Destination tubes labware 1 on 4, C2 of Destination tubes labware 1 on 4, D2 of Destination tubes labware 1 on 4, A3 of Destination tubes labware 1 on 4, B3 of Destination tubes labware 1 on 4, C3 of Destination tubes labware 1 on 4, D3 of Destination tubes labware 1 on 4, A4 of Destination tubes labware 1 on 4, B4 of Destination tubes labware 1 on 4, C4 of Destination tubes labware 1 on 4, D4 of Destination tubes labware 1 on 4, A5 of Destination tubes labware 1 on 4, B5 of Destination tubes labware 1 on 4, C5 of Destination tubes labware 1 on 4, D5 of Destination tubes labware 1 on 4, A6 of Destination tubes labware 1 on 4, B6 of Destination tubes labware 1 on 4, C6 of Destination tubes labware 1 on 4, D6 of Destination tubes labware 1 on 4]
B1 of buffer

In [12]:
max_trans_per_asp = 3  #230//(VOLUME_MMIX+5)
split_ind = [ind for ind in range(0, len(dest), max_trans_per_asp)]
dest_sets = [dest[split_ind[i]:split_ind[i+1]]
             for i in range(len(split_ind)-1)] + [dest[split_ind[-1]:]]

#dest_sets

In [ ]:
def retrieve_tip_info(pip,tipracks,file_path = '/data/A/tip_log.json'):
    tip_log = {}
    if tip_log and not robot.is_simulating():
        if os.path.isfile(file_path):
            with open(file_path) as json_file:
                data = json.load(json_file)
                if 'tips1000' in data:
                    tip_log['count'] = {pip: data['tips1000']}
                else:
                    tip_log['count'] = {pip: 0}
    else:
        tip_log['count'] = {pip: 0}

    tip_log['tips'] = {
        pip: [tip for rack in tipracks for tip in rack.wells()]}
    tip_log['max'] = {pip: len(tip_log['tips'][pip])}

    return tip_log

def save_tip_info(pip, file_path = '/data/A/tip_log.json'):
    if not robot.is_simulating():
        data = {'tips1000': tip_log['count'][pip]}
        with open(file_path, 'w') as outfile:
            json.dump(data, outfile)

def pick_up(pip,tiprack):
    ## retrieve tip_log
    global tip_log
    tip_log = retrieve_tip_info(pip,tiprack)
    if tip_log['count'][pip] == tip_log['max'][pip]:
        robot.pause('Replace ' + str(pip.max_volume) + 'µl tipracks before \
resuming.')
        pip.reset_tipracks()
        tip_log['count'][pip] = 0
    tip_log['count'][pip] += 1
    pip.pick_up_tip(tip_log['tips'][pip][tip_log['count'][pip]])
    
    
